In [ ]:
import sys
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import datetime

In [ ]:
from typing import List
import torch.nn as nn
import torch
import datetime


# Hyperparameters (Put here for now. Probably should find a way to organize them)
SIZE = 256
INPUT_CHANNELS = 3
OUTPUT_CHANNELS = 3
NUM_EPOCHS = 10
BATCH_SIZE = 5
LEARNING_RATE = 1e-3
BETA1 = 0.5 # Beta1 hyperparam for Adam optimizers
SHUFFLE = True


class NeuralModel(nn.Module):
    def __init__(self):
        super().__init__()

        # Load in default parameters
        self.size = SIZE
        self.input_c = INPUT_CHANNELS
        self.output_c = OUTPUT_CHANNELS
        self.num_epochs = NUM_EPOCHS
        self.batch_size = BATCH_SIZE
        self.lr = LEARNING_RATE
        self.beta1 = BETA1
        self.shuffle = SHUFFLE

        current_time = datetime.datetime.now() - datetime.timedelta(hours=8)
        print("Starting at:", current_time.strftime("%H:%M:%S"))


    def forward(self, obs):
        raise NotImplementedError()

    def train(self):
        raise NotImplementedError()

    def prepare_batch(self, data):
        pass

In [ ]:
import time
from typing import List
import functools
import torch.nn as nn
import torch
from torch.utils.data import DataLoader
from torch.utils import data
import h5py
import cv2


# Hyperparameters
NUM_DOWNS = 8
NGF = 64
LAMBDA_L1 = 100

class Dataset(data.Dataset):
    def __init__(self, x, y=None):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.x)

    def __getitem__(self, index):
        if self.y is None:
            return self.x[index]
        else:
            return self.x[index], self.y[index]


# Reference: https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix/
class Pix2PixUnet(NeuralModel, nn.Module):
    """Create a Unet-based generator"""

    def __init__(self):
        """Construct a Unet generator
        Parameters:
            input_nc (int)  -- the number of channels in input images
            output_nc (int) -- the number of channels in output images
            num_downs (int) -- the number of downsamplings in UNet. For example, # if |num_downs| == 7,
                                image of size 128x128 will become of size 1x1 # at the bottleneck
            ngf (int)       -- the number of filters in the last conv layer
            norm_layer      -- normalization layer
        We construct the U-Net from the innermost layer to the outermost layer.
        It is a recursive process.
        """
        super().__init__()

        # hyper params
        self.num_downs = NUM_DOWNS
        self.ngf = NGF
        self.lambda_L1 = LAMBDA_L1
        self.norm_layer = nn.BatchNorm2d

        # construct unet structure
        unet_block = UnetSkipConnectionBlock(self.ngf * 8, self.ngf * 8, input_nc=None, submodule=None, norm_layer=self.norm_layer, innermost=True)  # add the innermost layer
        for i in range(self.num_downs - 5):          # add intermediate layers with ngf * 8 filters
            unet_block = UnetSkipConnectionBlock(self.ngf * 8, self.ngf * 8, input_nc=None, submodule=unet_block, norm_layer=self.norm_layer, use_dropout=False)
        # gradually reduce the number of filters from ngf * 8 to ngf
        unet_block = UnetSkipConnectionBlock(self.ngf * 4, self.ngf * 8, input_nc=None, submodule=unet_block, norm_layer=self.norm_layer)
        unet_block = UnetSkipConnectionBlock(self.ngf * 2, self.ngf * 4, input_nc=None, submodule=unet_block, norm_layer=self.norm_layer)
        unet_block = UnetSkipConnectionBlock(self.ngf, self.ngf * 2, input_nc=None, submodule=unet_block, norm_layer=self.norm_layer)
        self.model = UnetSkipConnectionBlock(self.output_c, self.ngf, input_nc=self.input_c, submodule=unet_block, outermost=True, norm_layer=self.norm_layer)  # add the outermost layer
        
        
        self.device = torch.device(
            f"cuda:0" if torch.cuda.is_available() else "cpu"
        )
    
        print(f"Device: {self.device}")
        self.model.to(self.device)
        
        
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=self.lr, betas=(self.beta1, 0.999))
        self.criterion = nn.BCEWithLogitsLoss()
        self.criterionL1 = torch.nn.L1Loss()

    def forward(self, x):
        return self.model(x)


    def train(self):
        # load data from file
        with h5py.File("data_950_0to1.h5", 'r') as hf:
            data_x = hf['x'][:]
            data_y = hf['y'][:]

        # 80/20 split
        # data_tr, data_te = data_obs.split()
        train_ind = int(len(data_x) * 0.8)
        data_tr = Dataset(data_x[:train_ind], data_y[:train_ind])


        train_loader = DataLoader(
            data_tr,
            batch_size=self.batch_size,
            shuffle=self.shuffle
        )
        
        train_losses = []
        loss_gen_list = []
        loss_L1_list = []
        
        for epoch in range(self.num_epochs):
            start_time1 = time.time()

            epoch_loss = torch.tensor(0.0, device=self.device)

            for batch_idx, batch in enumerate(train_loader):
                start_time2 = time.time()

                data, target = batch

                outputs = self.model(data.float().to(self.device))
                
                # if you want to see the progress of the outputs
#                 if batch_idx % 20 == 0:
#                     out = outputs.detach().cpu().numpy()
#                     tar = target.detach().cpu().numpy()
#                     plt.subplot(1, 2, 1)
#                     plt.imshow(out.transpose((0, 2, 3, 1))[0])
#                     plt.subplot(1, 2, 2)
#                     plt.imshow(tar.transpose((0, 2, 3, 1))[0])
#                     plt.show()

                loss_gen = self.criterion(outputs, target.float().to(self.device))
                #loss_gen = 0
                loss_L1 = self.criterionL1(outputs, target.float().to(self.device)) * self.lambda_L1
                loss = loss_gen + loss_L1
                
                loss_gen_list.append(loss_gen)
                loss_L1_list.append(loss_L1)
                
                # Aggregate loss across mini-batches (per epoch)
                epoch_loss += loss

                # Backprop and perform Adam optimisation
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()
                if batch_idx % 38 == 0:
                    print(
                    f"Epoch: {epoch}\tBatch: {batch_idx}\tGen Loss: {loss_gen:.5f}\tL1 Loss: {loss_L1:.5f}"
                    f"\tTotal Time: {(time.time() - start_time2)/60:.3f} minutes"
                    )

            print(
                f"Epoch: {epoch}\tTrain Loss: {epoch_loss/len(train_loader):.5f}"
                f"\tTotal Time: {(time.time() - start_time1)/60:.3f} minutes"
            )

            train_losses.append(epoch_loss / len(train_loader))
        
        fig, (ax1, ax2) = plt.subplots(2, figsize=(15,15))
        fig.suptitle("Loss Plots")
        ax1.plot(range(len(loss_gen_list)), loss_gen_list, alpha = 0.7)
        ax1.set_title("Gen Loss")
        ax2.plot(range(len(loss_gen_list)), loss_L1_list, alpha = 0.7)
        ax2.set_title("L1 Loss")
        plt.show()
        
        return train_losses
    
    def predict(self):
        self.shuffle = False
        
        # load data from file
        with h5py.File("data_950_0to1.h5", 'r') as hf:
            data_x = hf['x'][:]
            data_y = hf['y'][:]
        
        color_distance = []
        outputImgs = []
        
        # 80/20 split
        # data_tr, data_te = data_obs.split()
        train_ind = int(len(data_x) * 0.8)
        #data_te = Dataset(data_x[train_ind:], data_y[train_ind:])
        data_te = Dataset(framesList, framesList)
        
        test_loader = DataLoader(
            data_te,
            batch_size=self.batch_size,
            shuffle=self.shuffle
        )
        
        for batch_idx, batch in enumerate(test_loader):
            start_time2 = time.time()
            #print(batch)
            data, target = batch
            x = data.detach().cpu().numpy().transpose((0, 2, 3, 1))[0]
            y = target.detach().cpu().numpy().transpose((0, 2, 3, 1))[0]
            
            outputs = self.model(data.float().to(self.device))
            
            y_hat = outputs.detach().cpu().numpy().transpose((0, 2, 3, 1))[0]
            
            dist = 0
            for i in range(0, len(y)):
                for j in range(0,len(y[i])):
                    dist += np.sqrt((y[i][j][0] - y_hat[i][j][0])**2 + (y[i][j][1] - y_hat[i][j][1])**2 +
                                      (y[i][j][2] - y_hat[i][j][2])**2)
            dist = dist/len(y)
            color_distance.append(dist)

            if True:
                '''
                fig, ax = plt.subplots(1, 3, figsize=(16, 5))
                ax[0].axis('off')
                ax[0].set_title('$X$')
                ax[0].imshow(x)
                ax[1].axis('off')
                ax[1].set_title('$\hat{Y}$')
                ax[1].imshow(y_hat)
                ax[2].axis('off')
                ax[2].set_title('$Y$')
                ax[2].imshow(y)   
                plt.show()
                '''
                
                plt.imshow(y_hat)
                #plt.savefig("data/MCFrame%d.png" % batch_idx)
                plt.show()
                
                print("Color difference: %d" % (dist))
                
            outputImgs.append(y_hat)
            
        print("Average color difference: %d" % (np.mean(np.array(color_distance))))
        print("Max color difference: %d" % (max(color_distance)))
        print("Min color difference: %d" % (min(color_distance)))
        
        print(color_distance)
        return outputImgs

class UnetSkipConnectionBlock(nn.Module):
    """Defines the Unet submodule with skip connection.
        X -------------------identity----------------------
        |-- downsampling -- |submodule| -- upsampling --|
    """

    def __init__(self, outer_nc, inner_nc, input_nc=None,
                 submodule=None, outermost=False, innermost=False, norm_layer=nn.BatchNorm2d, use_dropout=False):
        """Construct a Unet submodule with skip connections.
        Parameters:
            outer_nc (int) -- the number of filters in the outer conv layer
            inner_nc (int) -- the number of filters in the inner conv layer
            input_nc (int) -- the number of channels in input images/features
            submodule (UnetSkipConnectionBlock) -- previously defined submodules
            outermost (bool)    -- if this module is the outermost module
            innermost (bool)    -- if this module is the innermost module
            norm_layer          -- normalization layer
            use_dropout (bool)  -- if use dropout layers.
        """
        super(UnetSkipConnectionBlock, self).__init__()
        self.outermost = outermost
        if type(norm_layer) == functools.partial:
            use_bias = norm_layer.func == nn.InstanceNorm2d
        else:
            use_bias = norm_layer == nn.InstanceNorm2d
        if input_nc is None:
            input_nc = outer_nc
        downconv = nn.Conv2d(input_nc, inner_nc, kernel_size=4,
                             stride=2, padding=1, bias=use_bias)
        downrelu = nn.LeakyReLU(0.2, True)
        downnorm = norm_layer(inner_nc)
        uprelu = nn.ReLU(True)
        upnorm = norm_layer(outer_nc)

        if outermost:
            upconv = nn.ConvTranspose2d(inner_nc * 2, outer_nc,
                                        kernel_size=4, stride=2,
                                        padding=1)
            down = [downconv]
            up = [uprelu, upconv, nn.Tanh()]
            model = down + [submodule] + up
        elif innermost:
            upconv = nn.ConvTranspose2d(inner_nc, outer_nc,
                                        kernel_size=4, stride=2,
                                        padding=1, bias=use_bias)
            down = [downrelu, downconv]
            up = [uprelu, upconv, upnorm]
            model = down + up
        else:
            upconv = nn.ConvTranspose2d(inner_nc * 2, outer_nc,
                                        kernel_size=4, stride=2,
                                        padding=1, bias=use_bias)
            down = [downrelu, downconv, downnorm]
            up = [uprelu, upconv, upnorm]

            if use_dropout:
                model = down + [submodule] + up + [nn.Dropout(0.5)]
            else:
                model = down + [submodule] + up

        self.model = nn.Sequential(*model)

    def forward(self, x):
        if self.outermost:
            return self.model(x)
        else:   # add skip connections
            return torch.cat([x, self.model(x)], 1)

In [ ]:
model = Pix2PixUnet()

tr_losses = model.train()

In [ ]:
def crop_and_resize(img, size):
    #img = cv2.imread(filename)
    if img.shape[1] > img.shape[0]:
        start_y = int(img.shape[1]/2 - img.shape[0]/2)
        end_y = start_y + img.shape[0]
        crop = img[0:img.shape[0], start_y:end_y]
    else: 
        start_x = int(img.shape[0]/2 - img.shape[1]/2)
        end_x = start_x + img.shape[1]
        crop = img[start_x:end_x, 0:img.shape[1]]
    return cv2.resize(crop, (size, size))


In [ ]:
import cv2

vidcap = cv2.VideoCapture('data/MinecraftFootage1.mp4')
success,image = vidcap.read()
framesList = []
count = 0
while True:
    #count += 1
    #cv2.imwrite("data/frame%d.jpg" % count, image)     # save frame as JPEG file     
    success,image = vidcap.read()
    if not success: break
    #if count%2 != 0: continue
    image = crop_and_resize(image, 256)
    image[:,:,[2,0]] = image[:,:,[0,2]] #convert to RGB
    image = np.transpose(image, (2, 0, 1))
    image = image/255
    framesList.append(image)

framesList = np.asarray(framesList)


In [ ]:
np.shape(framesList)

In [ ]:
results = model.predict()  

In [ ]:
imgs = []

for i in range(0,270):
    #imgs.append(cv2.cvtColor(cv2.imread("data/MCFrame%d.png" % i, 1), cv2.COLOR_BGR2RGB))
    imgs.append(cv2.imread("data/MCFrame%d.png" % i, 1))

imgs = np.asarray(imgs)


In [ ]:
len(results)

In [ ]:
results[170]

In [ ]:
plt.imshow(results[170])

In [ ]:
plt.imshow((results[170]/2.0)+0.5, vmin=0,vmax=1)

In [ ]:
plt.imshow(np.abs(results[170])[:,:,[2,1,0]])

In [ ]:
for each in results:
    each /= 255

In [ ]:
plt.imshow(results[130])

In [ ]:
time = datetime.datetime.now().strftime("%H:%M:%S")
time = time.replace(':','-')
cv2out = cv2.VideoWriter('data/footageTestDirect'+time+'.mp4',cv2.VideoWriter_fourcc('m','p','4','v'), 12, (256,256))

for i in range(len(results)):
    cv2out.write((np.abs(results)[i]*255).astype('uint8')[:,:,[2,1,0]])
    print(i)
cv2out.release()

In [ ]:
import cv2

vidcap = cv2.VideoCapture('data/MinecraftFootage1.mp4')
success,image = vidcap.read()
framesListNEW = []
count = 0
while count<100:
    count += 1
    #cv2.imwrite("data/frame%d.jpg" % count, image)     # save frame as JPEG file     
    success,image = vidcap.read()
    if not success: break
    #if count%2 != 0: continue
    image = crop_and_resize(image, 256)
    #image[:,:,[2,0]] = image[:,:,[0,2]] #convert to RGB
    #image = np.transpose(image, (2, 0, 1))
    #image = image/255
    framesListNEW.append(image)




In [ ]:
np.shape(framesListNEW)

In [ ]:
plt.imshow(framesListNEW[0])

In [ ]:
cv2out = cv2.VideoWriter('data/OriginalFootageCropped.mp4',cv2.VideoWriter_fourcc('m','p','4','v'), 60, (256,256))
for f in framesList:
    cv2out.write(f)
cv2out.release()